# Hotel Bookings Analysis - TravClan Business Analyst Internship
## Technical Assignment

**Objective:** Analyze booking patterns, identify cancellation trends, and provide actionable business recommendations

**Dataset:** Hotel_bookings_final.csv

## 1. Setup and Data Loading

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Load the dataset
df = pd.read_csv('Hotel_bookings_final.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

## 2. Data Exploration and Preparation

In [ ]:
# Display basic information
print("First few rows:")
display(df.head())

print("\nData Types:")
display(df.dtypes)

print("\nMissing Values:")
display(df.isnull().sum())

print("\nBasic Statistics:")
display(df.describe())

In [ ]:
# Convert date columns to datetime
df['booking_date'] = pd.to_datetime(df['booking_date'])
df['check_in_date'] = pd.to_datetime(df['check_in_date'], errors='coerce')
df['check_out_date'] = pd.to_datetime(df['check_out_date'], errors='coerce')
df['travel_date'] = pd.to_datetime(df['travel_date'])

# Extract temporal features
df['booking_month'] = df['booking_date'].dt.month
df['booking_month_name'] = df['booking_date'].dt.month_name()
df['booking_day_of_week'] = df['booking_date'].dt.day_name()

# Calculate stay length for completed bookings
df['stay_length'] = (df['check_out_date'] - df['check_in_date']).dt.days

# Calculate lead time (days between booking and check-in)
df['lead_time'] = (df['check_in_date'] - df['booking_date']).dt.days

# Create cancellation flag
df['is_cancelled'] = (df['booking_status'] == 'Cancelled').astype(int)
df['is_failed'] = (df['booking_status'] == 'Failed').astype(int)
df['is_confirmed'] = (df['booking_status'] == 'Confirmed').astype(int)

print("Data preparation completed!")
print(f"\nBooking Status Distribution:")
print(df['booking_status'].value_counts())
print(f"\nCancellation Rate: {df['is_cancelled'].mean()*100:.2f}%")
print(f"Failure Rate: {df['is_failed'].mean()*100:.2f}%")
print(f"Confirmation Rate: {df['is_confirmed'].mean()*100:.2f}%")

## 3. KEY OBSERVATIONS - Identifying Trends and Patterns

### 3.1 Booking Status Overview

In [ ]:
# Overall booking status distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Pie chart
booking_counts = df['booking_status'].value_counts()
colors = ['#2ecc71', '#e74c3c', '#f39c12']
axes[0].pie(booking_counts, labels=booking_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90)
axes[0].set_title('Booking Status Distribution', fontsize=14, fontweight='bold')

# Bar chart with values
booking_counts.plot(kind='bar', ax=axes[1], color=colors)
axes[1].set_title('Booking Status Counts', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Booking Status')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

# Add value labels on bars
for i, v in enumerate(booking_counts):
    axes[1].text(i, v + 200, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('booking_status_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Total Bookings: {len(df):,}")
print(f"Cancellation Rate: {(df['is_cancelled'].sum()/len(df))*100:.2f}%")
print(f"Failure Rate: {(df['is_failed'].sum()/len(df))*100:.2f}%")

### 3.2 OBSERVATION 1: Booking Channel Performance Analysis

In [ ]:
# Booking channel analysis
channel_analysis = df.groupby('booking_channel').agg({
    'booking_status': 'count',
    'is_cancelled': ['sum', 'mean'],
    'is_confirmed': 'mean',
    'booking_value': 'mean',
    'selling_price': 'mean'
}).round(2)

channel_analysis.columns = ['Total_Bookings', 'Total_Cancellations', 'Cancellation_Rate', 
                              'Confirmation_Rate', 'Avg_Booking_Value', 'Avg_Selling_Price']
channel_analysis['Cancellation_Rate'] = (channel_analysis['Cancellation_Rate'] * 100).round(2)
channel_analysis['Confirmation_Rate'] = (channel_analysis['Confirmation_Rate'] * 100).round(2)
channel_analysis = channel_analysis.sort_values('Total_Bookings', ascending=False)

print("\n=== BOOKING CHANNEL PERFORMANCE ===")
display(channel_analysis)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Total bookings by channel
channel_analysis['Total_Bookings'].plot(kind='bar', ax=axes[0,0], color='steelblue')
axes[0,0].set_title('Total Bookings by Channel', fontsize=12, fontweight='bold')
axes[0,0].set_ylabel('Number of Bookings')
axes[0,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(channel_analysis['Total_Bookings']):
    axes[0,0].text(i, v + 100, str(v), ha='center')

# 2. Cancellation rate by channel
channel_analysis['Cancellation_Rate'].plot(kind='bar', ax=axes[0,1], color='coral')
axes[0,1].set_title('Cancellation Rate by Channel (%)', fontsize=12, fontweight='bold')
axes[0,1].set_ylabel('Cancellation Rate (%)')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', 
                   label=f'Overall Avg: {df["is_cancelled"].mean()*100:.1f}%')
axes[0,1].legend()
for i, v in enumerate(channel_analysis['Cancellation_Rate']):
    axes[0,1].text(i, v + 0.5, f"{v:.1f}%", ha='center')

# 3. Average booking value by channel
channel_analysis['Avg_Booking_Value'].plot(kind='bar', ax=axes[1,0], color='mediumseagreen')
axes[1,0].set_title('Average Booking Value by Channel', fontsize=12, fontweight='bold')
axes[1,0].set_ylabel('Average Booking Value (₹)')
axes[1,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(channel_analysis['Avg_Booking_Value']):
    axes[1,0].text(i, v + 100, f'₹{v:,.0f}', ha='center')

# 4. Confirmation rate by channel
channel_analysis['Confirmation_Rate'].plot(kind='bar', ax=axes[1,1], color='lightseagreen')
axes[1,1].set_title('Confirmation Rate by Channel (%)', fontsize=12, fontweight='bold')
axes[1,1].set_ylabel('Confirmation Rate (%)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].axhline(y=df['is_confirmed'].mean()*100, color='green', linestyle='--', 
                   label=f'Overall Avg: {df["is_confirmed"].mean()*100:.1f}%')
axes[1,1].legend()
for i, v in enumerate(channel_analysis['Confirmation_Rate']):
    axes[1,1].text(i, v + 1, f"{v:.1f}%", ha='center')

plt.tight_layout()
plt.savefig('booking_channel_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.3 OBSERVATION 2: Room Type and Star Rating Performance

In [ ]:
# Room type analysis
room_analysis = df.groupby('room_type').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean',
    'booking_value': 'mean',
    'stay_length': 'mean'
}).round(2)

room_analysis.columns = ['Total_Bookings', 'Cancellation_Rate', 'Avg_Booking_Value', 'Avg_Stay_Length']
room_analysis['Cancellation_Rate'] = (room_analysis['Cancellation_Rate'] * 100).round(2)
room_analysis = room_analysis.sort_values('Total_Bookings', ascending=False)

print("\n=== ROOM TYPE PERFORMANCE ===")
display(room_analysis)

# Star rating analysis
star_analysis = df.groupby('star_rating').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean',
    'booking_value': 'mean',
    'selling_price': 'mean'
}).round(2)

star_analysis.columns = ['Total_Bookings', 'Cancellation_Rate', 'Avg_Booking_Value', 'Avg_Selling_Price']
star_analysis['Cancellation_Rate'] = (star_analysis['Cancellation_Rate'] * 100).round(2)

print("\n=== STAR RATING PERFORMANCE ===")
display(star_analysis)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Bookings by room type
room_analysis['Total_Bookings'].plot(kind='bar', ax=axes[0,0], color='#3498db')
axes[0,0].set_title('Bookings by Room Type', fontsize=12, fontweight='bold')
axes[0,0].set_ylabel('Number of Bookings')
axes[0,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(room_analysis['Total_Bookings']):
    axes[0,0].text(i, v + 100, str(v), ha='center')

# 2. Cancellation rate by room type
room_analysis['Cancellation_Rate'].plot(kind='bar', ax=axes[0,1], color='#e74c3c')
axes[0,1].set_title('Cancellation Rate by Room Type (%)', fontsize=12, fontweight='bold')
axes[0,1].set_ylabel('Cancellation Rate (%)')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)
for i, v in enumerate(room_analysis['Cancellation_Rate']):
    axes[0,1].text(i, v + 0.5, f"{v:.1f}%", ha='center')

# 3. Bookings by star rating
star_analysis['Total_Bookings'].plot(kind='bar', ax=axes[1,0], color='#f39c12')
axes[1,0].set_title('Bookings by Star Rating', fontsize=12, fontweight='bold')
axes[1,0].set_xlabel('Star Rating')
axes[1,0].set_ylabel('Number of Bookings')
axes[1,0].tick_params(axis='x', rotation=0)
for i, v in enumerate(star_analysis['Total_Bookings']):
    axes[1,0].text(i, v + 100, str(v), ha='center')

# 4. Cancellation rate by star rating
star_analysis['Cancellation_Rate'].plot(kind='bar', ax=axes[1,1], color='#9b59b6')
axes[1,1].set_title('Cancellation Rate by Star Rating (%)', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Star Rating')
axes[1,1].set_ylabel('Cancellation Rate (%)')
axes[1,1].tick_params(axis='x', rotation=0)
axes[1,1].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)
for i, v in enumerate(star_analysis['Cancellation_Rate']):
    axes[1,1].text(i, v + 0.5, f"{v:.1f}%", ha='center')

plt.tight_layout()
plt.savefig('room_star_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.4 OBSERVATION 3: Cancellation Behavior Deep Dive

In [ ]:
# Cancellation analysis by multiple factors
print("\n=== DETAILED CANCELLATION ANALYSIS ===")

# By payment method
payment_cancellation = df.groupby('payment_method').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean'
}).round(3)
payment_cancellation.columns = ['Total_Bookings', 'Cancellation_Rate']
payment_cancellation['Cancellation_Rate'] = (payment_cancellation['Cancellation_Rate'] * 100).round(2)
payment_cancellation = payment_cancellation.sort_values('Cancellation_Rate', ascending=False)

print("\nCancellation Rate by Payment Method:")
display(payment_cancellation)

# By stay type
stay_cancellation = df.groupby('stay_type').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean',
    'booking_value': 'mean'
}).round(3)
stay_cancellation.columns = ['Total_Bookings', 'Cancellation_Rate', 'Avg_Booking_Value']
stay_cancellation['Cancellation_Rate'] = (stay_cancellation['Cancellation_Rate'] * 100).round(2)

print("\nCancellation Rate by Stay Type:")
display(stay_cancellation)

# Lead time analysis for cancelled bookings
cancelled_df = df[df['is_cancelled'] == 1].copy()
confirmed_df = df[df['is_confirmed'] == 1].copy()

print(f"\nAverage Lead Time for Cancelled Bookings: {cancelled_df['lead_time'].mean():.1f} days")
print(f"Average Lead Time for Confirmed Bookings: {confirmed_df['lead_time'].mean():.1f} days")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Cancellation by payment method
payment_cancellation['Cancellation_Rate'].plot(kind='barh', ax=axes[0,0], color='#e67e22')
axes[0,0].set_title('Cancellation Rate by Payment Method', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Cancellation Rate (%)')
axes[0,0].axvline(x=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)
for i, v in enumerate(payment_cancellation['Cancellation_Rate']):
    axes[0,0].text(v + 0.5, i, f"{v:.1f}%", va='center')

# 2. Cancellation by stay type
stay_cancellation['Cancellation_Rate'].plot(kind='bar', ax=axes[0,1], color='#16a085')
axes[0,1].set_title('Cancellation Rate by Stay Type', fontsize=12, fontweight='bold')
axes[0,1].set_ylabel('Cancellation Rate (%)')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)
for i, v in enumerate(stay_cancellation['Cancellation_Rate']):
    axes[0,1].text(i, v + 0.5, f"{v:.1f}%", ha='center')

# 3. Lead time distribution comparison
axes[1,0].hist([confirmed_df['lead_time'].dropna(), cancelled_df['lead_time'].dropna()], 
               bins=30, label=['Confirmed', 'Cancelled'], alpha=0.7, color=['green', 'red'])
axes[1,0].set_title('Lead Time Distribution: Confirmed vs Cancelled', fontsize=12, fontweight='bold')
axes[1,0].set_xlabel('Lead Time (days)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].legend()

# 4. Booking value vs cancellation
booking_value_bins = pd.cut(df['booking_value'], bins=5)
value_cancellation = df.groupby(booking_value_bins)['is_cancelled'].mean() * 100
value_cancellation.plot(kind='bar', ax=axes[1,1], color='#8e44ad')
axes[1,1].set_title('Cancellation Rate by Booking Value Range', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Booking Value Range')
axes[1,1].set_ylabel('Cancellation Rate (%)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('cancellation_deep_dive.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.5 Temporal Trends and Seasonality

In [ ]:
# Monthly booking trends
monthly_trends = df.groupby('booking_month').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean',
    'booking_value': 'mean',
    'stay_length': 'mean'
}).round(2)

monthly_trends.columns = ['Total_Bookings', 'Cancellation_Rate', 'Avg_Booking_Value', 'Avg_Stay_Length']
monthly_trends['Cancellation_Rate'] = (monthly_trends['Cancellation_Rate'] * 100).round(2)

# Day of week analysis
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_trends = df.groupby('booking_day_of_week').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean',
    'booking_value': 'mean'
}).round(2)
dow_trends.columns = ['Total_Bookings', 'Cancellation_Rate', 'Avg_Booking_Value']
dow_trends['Cancellation_Rate'] = (dow_trends['Cancellation_Rate'] * 100).round(2)
dow_trends = dow_trends.reindex(dow_order)

print("\n=== TEMPORAL TRENDS ===")
print("\nMonthly Booking Trends:")
display(monthly_trends)
print("\nDay of Week Trends:")
display(dow_trends)

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Monthly bookings
axes[0,0].plot(monthly_trends.index, monthly_trends['Total_Bookings'], marker='o', 
               linewidth=2, markersize=8, color='#3498db')
axes[0,0].set_title('Monthly Booking Volume', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Month')
axes[0,0].set_ylabel('Number of Bookings')
axes[0,0].grid(True, alpha=0.3)
for i, v in enumerate(monthly_trends['Total_Bookings']):
    axes[0,0].text(monthly_trends.index[i], v + 50, str(v), ha='center')

# 2. Monthly cancellation rate
axes[0,1].plot(monthly_trends.index, monthly_trends['Cancellation_Rate'], marker='s', 
               linewidth=2, markersize=8, color='#e74c3c')
axes[0,1].set_title('Monthly Cancellation Rate', fontsize=12, fontweight='bold')
axes[0,1].set_xlabel('Month')
axes[0,1].set_ylabel('Cancellation Rate (%)')
axes[0,1].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)
axes[0,1].grid(True, alpha=0.3)

# 3. Day of week bookings
dow_trends['Total_Bookings'].plot(kind='bar', ax=axes[1,0], color='#2ecc71')
axes[1,0].set_title('Bookings by Day of Week', fontsize=12, fontweight='bold')
axes[1,0].set_xlabel('Day of Week')
axes[1,0].set_ylabel('Number of Bookings')
axes[1,0].tick_params(axis='x', rotation=45)
for i, v in enumerate(dow_trends['Total_Bookings']):
    axes[1,0].text(i, v + 20, str(int(v)), ha='center')

# 4. Average booking value over time
axes[1,1].plot(monthly_trends.index, monthly_trends['Avg_Booking_Value'], marker='D', 
               linewidth=2, markersize=8, color='#f39c12')
axes[1,1].set_title('Average Booking Value by Month', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Month')
axes[1,1].set_ylabel('Average Booking Value (₹)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('temporal_trends.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.6 City-wise Performance Analysis

In [ ]:
# City analysis
city_analysis = df.groupby('city').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean',
    'booking_value': 'mean',
    'stay_length': 'mean'
}).round(2)

city_analysis.columns = ['Total_Bookings', 'Cancellation_Rate', 'Avg_Booking_Value', 'Avg_Stay_Length']
city_analysis['Cancellation_Rate'] = (city_analysis['Cancellation_Rate'] * 100).round(2)
city_analysis = city_analysis.sort_values('Total_Bookings', ascending=False)

print("\n=== CITY-WISE PERFORMANCE ===")
display(city_analysis)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Bookings by city
city_analysis['Total_Bookings'].plot(kind='barh', ax=axes[0], color='#1abc9c')
axes[0].set_title('Bookings by City', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Number of Bookings')
for i, v in enumerate(city_analysis['Total_Bookings']):
    axes[0].text(v + 50, i, str(v), va='center')

# 2. Cancellation rate by city
city_analysis['Cancellation_Rate'].plot(kind='barh', ax=axes[1], color='#e67e22')
axes[1].set_title('Cancellation Rate by City (%)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Cancellation Rate (%)')
axes[1].axvline(x=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)
for i, v in enumerate(city_analysis['Cancellation_Rate']):
    axes[1].text(v + 0.3, i, f"{v:.1f}%", va='center')

plt.tight_layout()
plt.savefig('city_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.7 Promotional and Pricing Analysis

In [ ]:
# Coupon usage analysis
coupon_analysis = df.groupby('Coupon USed?').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean',
    'booking_value': 'mean',
    'coupon_redeem': 'mean'
}).round(2)

coupon_analysis.columns = ['Total_Bookings', 'Cancellation_Rate', 'Avg_Booking_Value', 'Avg_Coupon_Value']
coupon_analysis['Cancellation_Rate'] = (coupon_analysis['Cancellation_Rate'] * 100).round(2)

print("\n=== COUPON USAGE ANALYSIS ===")
display(coupon_analysis)

# Refund analysis
refund_analysis = df.groupby('refund_status').agg({
    'booking_status': 'count',
    'is_cancelled': 'mean',
    'refund_amount': 'mean'
}).round(2)

refund_analysis.columns = ['Total_Bookings', 'Cancellation_Rate', 'Avg_Refund_Amount']
refund_analysis['Cancellation_Rate'] = (refund_analysis['Cancellation_Rate'] * 100).round(2)

print("\nRefund Status Analysis:")
display(refund_analysis)

# Calculate markup percentage
df['markup_percentage'] = (df['markup'] / df['costprice'] * 100).round(2)

print(f"\nAverage Markup Percentage: {df['markup_percentage'].mean():.2f}%")
print(f"Median Markup Percentage: {df['markup_percentage'].median():.2f}%")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Coupon usage impact
coupon_analysis['Cancellation_Rate'].plot(kind='bar', ax=axes[0,0], color=['#e74c3c', '#2ecc71'])
axes[0,0].set_title('Cancellation Rate: Coupon Used vs Not Used', fontsize=12, fontweight='bold')
axes[0,0].set_ylabel('Cancellation Rate (%)')
axes[0,0].tick_params(axis='x', rotation=0)
axes[0,0].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)
for i, v in enumerate(coupon_analysis['Cancellation_Rate']):
    axes[0,0].text(i, v + 0.5, f"{v:.1f}%", ha='center')

# 2. Booking value distribution
axes[0,1].hist(df['booking_value'], bins=50, color='#3498db', edgecolor='black', alpha=0.7)
axes[0,1].set_title('Booking Value Distribution', fontsize=12, fontweight='bold')
axes[0,1].set_xlabel('Booking Value (₹)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].axvline(df['booking_value'].mean(), color='red', linestyle='--', 
                   label=f'Mean: ₹{df["booking_value"].mean():,.0f}')
axes[0,1].legend()

# 3. Markup percentage distribution
axes[1,0].hist(df['markup_percentage'], bins=50, color='#f39c12', edgecolor='black', alpha=0.7)
axes[1,0].set_title('Markup Percentage Distribution', fontsize=12, fontweight='bold')
axes[1,0].set_xlabel('Markup Percentage (%)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].axvline(df['markup_percentage'].mean(), color='red', linestyle='--', 
                   label=f'Mean: {df["markup_percentage"].mean():.1f}%')
axes[1,0].legend()

# 4. Cashback distribution
cashback_data = df[df['cashback'] > 0]['cashback']
axes[1,1].hist(cashback_data, bins=30, color='#9b59b6', edgecolor='black', alpha=0.7)
axes[1,1].set_title('Cashback Distribution (Where Applicable)', fontsize=12, fontweight='bold')
axes[1,1].set_xlabel('Cashback Amount (₹)')
axes[1,1].set_ylabel('Frequency')
axes[1,1].axvline(cashback_data.mean(), color='red', linestyle='--', 
                   label=f'Mean: ₹{cashback_data.mean():.2f}')
axes[1,1].legend()

plt.tight_layout()
plt.savefig('promotional_pricing_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. ROOT CAUSE ANALYSIS

### 4.1 Cancellation Pattern Analysis

In [ ]:
# Create a comprehensive cancellation factors dataframe
cancelled_bookings = df[df['is_cancelled'] == 1].copy()
confirmed_bookings = df[df['is_confirmed'] == 1].copy()

print("\n" + "="*80)
print("ROOT CAUSE ANALYSIS: CANCELLATION PATTERNS")
print("="*80)

# Factor 1: Lead Time Impact
print("\n1. LEAD TIME IMPACT:")
print(f"   Average lead time for cancelled bookings: {cancelled_bookings['lead_time'].mean():.1f} days")
print(f"   Average lead time for confirmed bookings: {confirmed_bookings['lead_time'].mean():.1f} days")
print(f"   Difference: {abs(cancelled_bookings['lead_time'].mean() - confirmed_bookings['lead_time'].mean()):.1f} days")
print("   → Insight: Longer lead times are associated with higher cancellation risk")

# Factor 2: Booking Value Impact
print("\n2. BOOKING VALUE IMPACT:")
print(f"   Average booking value for cancelled: ₹{cancelled_bookings['booking_value'].mean():,.2f}")
print(f"   Average booking value for confirmed: ₹{confirmed_bookings['booking_value'].mean():,.2f}")
high_value_cancel = df[df['booking_value'] > df['booking_value'].quantile(0.75)]['is_cancelled'].mean() * 100
low_value_cancel = df[df['booking_value'] <= df['booking_value'].quantile(0.25)]['is_cancelled'].mean() * 100
print(f"   Cancellation rate for high-value bookings (top 25%): {high_value_cancel:.2f}%")
print(f"   Cancellation rate for low-value bookings (bottom 25%): {low_value_cancel:.2f}%")

# Factor 3: Payment Method Impact
print("\n3. PAYMENT METHOD INSIGHTS:")
payment_risk = df.groupby('payment_method')['is_cancelled'].mean().sort_values(ascending=False) * 100
for method, rate in payment_risk.items():
    print(f"   {method}: {rate:.2f}% cancellation rate")
print("   → Insight: Payment method flexibility may influence cancellation likelihood")

# Factor 4: Channel Performance
print("\n4. CHANNEL PERFORMANCE DIFFERENCES:")
channel_risk = df.groupby('booking_channel')['is_cancelled'].mean().sort_values(ascending=False) * 100
for channel, rate in channel_risk.items():
    print(f"   {channel}: {rate:.2f}% cancellation rate")

# Factor 5: Stay Type Analysis
print("\n5. STAY TYPE PATTERNS:")
for stay_type in df['stay_type'].unique():
    cancel_rate = df[df['stay_type'] == stay_type]['is_cancelled'].mean() * 100
    avg_value = df[df['stay_type'] == stay_type]['booking_value'].mean()
    print(f"   {stay_type}: {cancel_rate:.2f}% cancellation, Avg Value: ₹{avg_value:,.0f}")

In [ ]:
# Correlation analysis
print("\n" + "="*80)
print("CORRELATION ANALYSIS WITH CANCELLATION")
print("="*80)

# Select numeric columns for correlation
numeric_cols = ['booking_value', 'costprice', 'markup', 'selling_price', 
                'num_rooms_booked', 'stay_length', 'lead_time', 'cashback', 
                'coupon_redeem', 'markup_percentage', 'is_cancelled']

correlation_data = df[numeric_cols].corr()['is_cancelled'].sort_values(ascending=False)
print("\nCorrelation with Cancellation:")
for col, corr in correlation_data.items():
    if col != 'is_cancelled':
        print(f"   {col:25s}: {corr:7.4f}")

# Visualization of key factors
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Lead time vs cancellation
lead_time_bins = pd.cut(df['lead_time'].dropna(), bins=10)
lead_cancel = df.groupby(lead_time_bins)['is_cancelled'].mean() * 100
lead_cancel.plot(kind='bar', ax=axes[0,0], color='#e74c3c')
axes[0,0].set_title('Cancellation Rate by Lead Time', fontsize=12, fontweight='bold')
axes[0,0].set_xlabel('Lead Time (days)')
axes[0,0].set_ylabel('Cancellation Rate (%)')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)

# 2. Booking value vs cancellation
value_bins = pd.qcut(df['booking_value'], q=10, duplicates='drop')
value_cancel = df.groupby(value_bins)['is_cancelled'].mean() * 100
value_cancel.plot(kind='bar', ax=axes[0,1], color='#3498db')
axes[0,1].set_title('Cancellation Rate by Booking Value Decile', fontsize=12, fontweight='bold')
axes[0,1].set_xlabel('Booking Value Range')
axes[0,1].set_ylabel('Cancellation Rate (%)')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)

# 3. Number of rooms vs cancellation
rooms_cancel = df.groupby('num_rooms_booked')['is_cancelled'].mean() * 100
rooms_cancel.plot(kind='bar', ax=axes[1,0], color='#f39c12')
axes[1,0].set_title('Cancellation Rate by Number of Rooms', fontsize=12, fontweight='bold')
axes[1,0].set_xlabel('Number of Rooms Booked')
axes[1,0].set_ylabel('Cancellation Rate (%)')
axes[1,0].axhline(y=df['is_cancelled'].mean()*100, color='red', linestyle='--', alpha=0.7)

# 4. Correlation heatmap of top factors
top_factors = ['is_cancelled', 'lead_time', 'booking_value', 'num_rooms_booked', 
               'markup_percentage', 'cashback']
corr_matrix = df[top_factors].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdYlGn_r', center=0, 
            ax=axes[1,1], cbar_kws={'label': 'Correlation'})
axes[1,1].set_title('Correlation Heatmap: Key Cancellation Factors', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('root_cause_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. KEY INSIGHTS SUMMARY

In [ ]:
print("\n" + "="*80)
print("KEY INSIGHTS SUMMARY")
print("="*80)

print("\n📊 OVERALL METRICS:")
print(f"   • Total Bookings: {len(df):,}")
print(f"   • Confirmed: {df['is_confirmed'].sum():,} ({df['is_confirmed'].mean()*100:.1f}%)")
print(f"   • Cancelled: {df['is_cancelled'].sum():,} ({df['is_cancelled'].mean()*100:.1f}%)")
print(f"   • Failed: {df['is_failed'].sum():,} ({df['is_failed'].mean()*100:.1f}%)")
print(f"   • Average Booking Value: ₹{df['booking_value'].mean():,.2f}")
print(f"   • Total Revenue (Confirmed): ₹{df[df['is_confirmed']==1]['selling_price'].sum():,.2f}")

print("\n🎯 TOP 3 MEANINGFUL TRENDS:")
print("\n1. CHANNEL DISPARITY:")
best_channel = channel_analysis['Cancellation_Rate'].idxmin()
worst_channel = channel_analysis['Cancellation_Rate'].idxmax()
print(f"   • Best performing channel: {best_channel} ({channel_analysis.loc[best_channel, 'Cancellation_Rate']:.1f}% cancellation)")
print(f"   • Worst performing channel: {worst_channel} ({channel_analysis.loc[worst_channel, 'Cancellation_Rate']:.1f}% cancellation)")
print(f"   • Difference: {channel_analysis.loc[worst_channel, 'Cancellation_Rate'] - channel_analysis.loc[best_channel, 'Cancellation_Rate']:.1f} percentage points")

print("\n2. ROOM TYPE PREFERENCES:")
print(f"   • Most booked: {room_analysis.index[0]} ({room_analysis.iloc[0]['Total_Bookings']} bookings)")
print(f"   • Highest value: Suite (₹{room_analysis.loc['Suite', 'Avg_Booking_Value']:,.0f} avg)")
print(f"   • Lowest cancellation: {room_analysis['Cancellation_Rate'].idxmin()} ({room_analysis['Cancellation_Rate'].min():.1f}%)")

print("\n3. PAYMENT METHOD RISK:")
print(f"   • Highest risk: {payment_cancellation.index[0]} ({payment_cancellation.iloc[0]['Cancellation_Rate']:.1f}% cancellation)")
print(f"   • Lowest risk: {payment_cancellation.index[-1]} ({payment_cancellation.iloc[-1]['Cancellation_Rate']:.1f}% cancellation)")

print("\n💡 CRITICAL BUSINESS INSIGHTS:")
print("\n   A. Cancellation Risk Factors:")
print(f"      • Long lead times (>{cancelled_bookings['lead_time'].mean():.0f} days) show {(cancelled_bookings['lead_time'] > 30).sum() / len(cancelled_bookings) * 100:.1f}% of cancellations")
print(f"      • Business travel shows {stay_cancellation.loc['Business', 'Cancellation_Rate']:.1f}% vs Leisure {stay_cancellation.loc['Leisure', 'Cancellation_Rate']:.1f}%")
print(f"      • Mobile App bookings have {channel_analysis.loc['Mobile App', 'Cancellation_Rate']:.1f}% cancellation rate")

print("\n   B. Revenue Optimization Opportunities:")
lost_revenue = cancelled_bookings['selling_price'].sum()
potential_revenue = df[df['is_confirmed']==1]['selling_price'].sum()
print(f"      • Lost revenue from cancellations: ₹{lost_revenue:,.2f}")
print(f"      • Potential revenue increase if 50% cancellations prevented: ₹{lost_revenue*0.5:,.2f}")
print(f"      • Average markup: {df['markup_percentage'].mean():.1f}%")

print("\n   C. Customer Behavior Patterns:")
print(f"      • Coupon users have {coupon_analysis.loc['Yes', 'Cancellation_Rate']:.1f}% cancellation vs {coupon_analysis.loc['No', 'Cancellation_Rate']:.1f}% non-users")
print(f"      • Average stay length: {df['stay_length'].mean():.1f} nights")
print(f"      • {(df['num_rooms_booked'] > 1).sum()} bookings ({(df['num_rooms_booked'] > 1).sum()/len(df)*100:.1f}%) are for multiple rooms")

## 6. BUSINESS RECOMMENDATIONS

In [ ]:
print("\n" + "="*80)
print("ACTIONABLE BUSINESS RECOMMENDATIONS")
print("="*80)

print("\n🎯 PRIORITY 1: REDUCE CANCELLATIONS (Target: Reduce by 30%)")
print("\n   Strategy A: Lead Time Management")
print("   ────────────────────────────────")
print("   1. Implement graduated deposit structure:")
print(f"      • Bookings >45 days: 30% non-refundable deposit")
print(f"      • Bookings 30-45 days: 20% deposit")
print(f"      • Bookings <30 days: 10% deposit")
print(f"   2. Expected Impact: ~15-20% reduction in cancellations")
print(f"      • Potential revenue protection: ₹{lost_revenue*0.175:,.2f}")

print("\n   Strategy B: Payment Method Optimization")
print("   ──────────────────────────────────────")
print(f"   1. Incentivize low-risk payment methods ({payment_cancellation.index[-1]})")
print("   2. Add 2% cashback for Bank Transfer/Debit Card payments")
print("   3. Implement payment plan options for high-value bookings (>₹20,000)")
print(f"   4. Expected Impact: ~8-10% reduction in cancellations")

print("\n   Strategy C: Channel-Specific Interventions")
print("   ────────────────────────────────────")
best_3_channels = channel_analysis.nsmallest(3, 'Cancellation_Rate').index.tolist()
print(f"   1. Focus marketing on best-performing channels: {', '.join(best_3_channels)}")
print(f"   2. For Travel Agent bookings (highest cancellation):")
print("      • Require agent commission holdback until check-in")
print("      • Implement agent performance scoring")
print("   3. Expected Impact: ~5-7% improvement in overall confirmation rate")

print("\n💰 PRIORITY 2: IMPROVE PROFITABILITY (Target: +15% revenue)")
print("\n   Strategy A: Dynamic Pricing Optimization")
print("   ──────────────────────────────────────")
print("   1. Implement surge pricing for:")
print(f"      • Peak booking days (identified: {dow_trends['Total_Bookings'].idxmax()})")
print(f"      • High-demand cities (Top 3: {', '.join(city_analysis.index[:3].tolist())})")
print(f"      • Last-minute bookings (<7 days lead time)")
print(f"   2. Current avg markup: {df['markup_percentage'].mean():.1f}% → Target: 33%")
print(f"   3. Potential revenue increase: ₹{(df[df['is_confirmed']==1]['selling_price'].sum() * 0.05):,.2f}")

print("\n   Strategy B: Upselling & Cross-selling")
print("   ────────────────────────────────")
standard_users = df[df['room_type'] == 'Standard']['customer_id'].nunique()
upgrade_potential = standard_users * 0.15  # Assume 15% conversion
avg_upgrade_value = room_analysis.loc['Deluxe', 'Avg_Booking_Value'] - room_analysis.loc['Standard', 'Avg_Booking_Value']
print("   1. Room upgrade campaigns:")
print(f"      • Target Standard room bookers with Deluxe upgrades")
print(f"      • Offer 'upgrade for ₹2000 more' at booking")
print(f"      • Potential: {upgrade_potential:.0f} upgrades = ₹{upgrade_potential * avg_upgrade_value:,.0f} additional revenue")
print("   2. Add-on services: Airport transfers, breakfast packages, spa")
print(f"      • Target: 20% attachment rate, avg ₹1500 per add-on")

print("\n   Strategy C: Premium Tier Program")
print("   ──────────────────────────────")
high_value_customers = df[df['booking_value'] > df['booking_value'].quantile(0.75)]['customer_id'].nunique()
print(f"   1. Create 'TravClan Elite' membership (₹999/year)")
print(f"      • Target {high_value_customers} high-value customers")
print("      • Benefits: 5% instant discount, priority support, free cancellation (1 per year)")
print(f"      • Revenue potential: ₹{high_value_customers * 999 * 0.3:,.0f} (assuming 30% conversion)")

print("\n🔄 PRIORITY 3: INCREASE REPEAT BOOKINGS (Target: +25% repeat rate)")
print("\n   Strategy A: Loyalty Program")
print("   ──────────────────────────")
print("   1. Points-based system: 1 point per ₹100 spent")
print("   2. Redemption: 1000 points = ₹500 discount")
print("   3. Tier benefits:")
print("      • Silver (2 bookings): 5% bonus points")
print("      • Gold (5 bookings): 10% bonus points + free upgrade (subject to availability)")
print("      • Platinum (10+ bookings): 15% bonus points + guaranteed upgrade")
current_repeat_potential = len(df['customer_id'].unique())
print(f"   4. Target: Convert {current_repeat_potential * 0.25:.0f} customers to repeat bookers")

print("\n   Strategy B: Post-Stay Engagement")
print("   ──────────────────────────────")
print("   1. Automated email campaign:")
print("      • Day 3 post-checkout: Review request + ₹200 next-booking voucher")
print("      • Day 30: 'We miss you' with 10% discount code")
print("      • Day 90: Personalized destination recommendations")
print("   2. Birthday/Anniversary offers (15% off)")
print(f"   3. Expected: 15-20% increase in repeat bookings within 6 months")

print("\n   Strategy C: Referral Program")
print("   ──────────────────────────")
print("   1. Give ₹500, Get ₹500 program")
print("      • Referrer gets ₹500 credit after friend's first stay")
print("      • Referee gets ₹500 off first booking >₹5000")
print(f"   2. Target: 10% customer acquisition through referrals")
print(f"      • Estimated 3,000 new customers at lower CAC")

print("\n📈 EXPECTED OVERALL IMPACT (6-MONTH PROJECTION)")
print("   ─────────────────────────────────────────────")
current_revenue = df[df['is_confirmed']==1]['selling_price'].sum()
print(f"   • Cancellation reduction (30%): +₹{lost_revenue*0.3:,.2f}")
print(f"   • Profitability improvements (15%): +₹{current_revenue*0.15:,.2f}")
print(f"   • Repeat booking increase (25%): +₹{current_revenue*0.125:,.2f}")
print(f"   • TOTAL PROJECTED REVENUE INCREASE: +₹{(lost_revenue*0.3 + current_revenue*0.275):,.2f}")
print(f"   • Percentage increase: +{((lost_revenue*0.3 + current_revenue*0.275)/current_revenue)*100:.1f}%")

## 7. IMPLEMENTATION ROADMAP

In [ ]:
print("\n" + "="*80)
print("90-DAY IMPLEMENTATION ROADMAP")
print("="*80)

print("\n📅 PHASE 1: IMMEDIATE (Days 1-30) - Quick Wins")
print("   ─────────────────────────────────────────")
print("   ✓ Week 1-2: Deposit Policy")
print("     • Implement graduated deposit structure")
print("     • Update booking flow and T&Cs")
print("     • Team training on new policies")
print("\n   ✓ Week 2-3: Payment Incentives")
print("     • Launch cashback for preferred payment methods")
print("     • Update checkout page with incentive messaging")
print("\n   ✓ Week 3-4: Email Campaigns")
print("     • Set up post-stay engagement sequences")
print("     • Create voucher system for reviews")

print("\n📅 PHASE 2: SHORT-TERM (Days 31-60) - Core Programs")
print("   ───────────────────────────────────────────────")
print("   ✓ Week 5-6: Loyalty Program Launch")
print("     • Build points tracking system")
print("     • Create tier structure and benefits")
print("     • Launch marketing campaign")
print("\n   ✓ Week 6-7: Dynamic Pricing")
print("     • Implement basic surge pricing rules")
print("     • A/B test pricing strategies")
print("\n   ✓ Week 7-8: Channel Optimization")
print("     • Negotiate better terms with high-performing channels")
print("     • Implement agent performance tracking")

print("\n📅 PHASE 3: MEDIUM-TERM (Days 61-90) - Advanced Features")
print("   ────────────────────────────────────────────────────")
print("   ✓ Week 9-10: Upselling System")
print("     • Add room upgrade options at booking")
print("     • Implement add-on service marketplace")
print("\n   ✓ Week 10-11: Premium Tier")
print("     • Launch TravClan Elite membership")
print("     • Set up premium customer support")
print("\n   ✓ Week 11-12: Referral Program")
print("     • Build referral tracking system")
print("     • Create referral landing pages")
print("     • Launch referral campaign")

print("\n📊 SUCCESS METRICS & KPIs")
print("   ────────────────────────")
print("   Track Weekly:")
print(f"   • Cancellation rate (Target: <{df['is_cancelled'].mean()*100*0.7:.1f}%)")
print(f"   • Average booking value (Target: >₹{df['booking_value'].mean()*1.1:,.0f})")
print("   • Repeat booking rate (Target: >25%)")
print("   • Revenue per booking (Target: +15%)")
print("   • Customer satisfaction score (Target: >4.5/5)")

print("\n✅ NEXT STEPS FOR TEAM")
print("   ────────────────────")
print("   1. Review and approve roadmap")
print("   2. Assign ownership for each initiative")
print("   3. Set up monitoring dashboards")
print("   4. Schedule weekly review meetings")
print("   5. Prepare change management communication")

## 8. EXPORT RESULTS

In [ ]:
# Save key insights to CSV for reference
print("\nExporting analysis results...")

# Export channel analysis
channel_analysis.to_csv('channel_performance.csv')
print("✓ Saved: channel_performance.csv")

# Export room type analysis
room_analysis.to_csv('room_type_performance.csv')
print("✓ Saved: room_type_performance.csv")

# Export star rating analysis
star_analysis.to_csv('star_rating_performance.csv')
print("✓ Saved: star_rating_performance.csv")

# Export city analysis
city_analysis.to_csv('city_performance.csv')
print("✓ Saved: city_performance.csv")

# Export monthly trends
monthly_trends.to_csv('monthly_trends.csv')
print("✓ Saved: monthly_trends.csv")

print("\n" + "="*80)
print("ANALYSIS COMPLETE!")
print("="*80)
print("\nGenerated Files:")
print("   📊 Visualizations:")
print("      • booking_status_overview.png")
print("      • booking_channel_analysis.png")
print("      • room_star_analysis.png")
print("      • cancellation_deep_dive.png")
print("      • temporal_trends.png")
print("      • city_analysis.png")
print("      • promotional_pricing_analysis.png")
print("      • root_cause_analysis.png")
print("\n   📈 Data Exports:")
print("      • channel_performance.csv")
print("      • room_type_performance.csv")
print("      • star_rating_performance.csv")
print("      • city_performance.csv")
print("      • monthly_trends.csv")
print("\n🎯 Ready for presentation creation!")